In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import os

import re
import requests
import datetime
from datetime import datetime
from bs4 import BeautifulSoup
import bs4.element
from tqdm.notebook import tqdm
from konlpy.tag import Komoran

from sklearn.manifold import TSNE
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models.word2vec import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from konlpy.tag import Okt
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')

C:\Users\Hyeongbin\anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  return safe_unichr(cp)


# Crawling 

In [17]:
date = str(datetime.now())
date = date[:date.rfind(':')].replace(' ', '_')
date = date.replace(':','시') + '분'
today = str(datetime.now().strftime('%Y%m%d'))
print(today)

def get_soup_obj(url):
    headers = { "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36" }
    res = requests.get(url, headers = headers)
    soup = BeautifulSoup(res.text,'lxml')
    
    return soup

def get_news_contents(url):
    soup = get_soup_obj(url)
    body = soup.find('div', class_="_article_body_contents")

    news_contents = ''
    for content in body:
        if type(content) is bs4.element.NavigableString and len(content) > 50:
            news_contents += content.strip() + ' '

    return news_contents

def get_news_info(url, s) : 
    default_img = "https://search.naver.com/search.naver?where=image&sm=tab_jum&query=naver#"
    current_page = 1     
    news_info_list = []

    for i in range (10) : 
        sec_url = url + s + "&date=" + today + "&page=" + str(current_page)
        soup = get_soup_obj(sec_url)
        lis = soup.find('ul', class_='type06_headline').find_all("li", limit=15)
        
        for li in lis : 
            news_info = {
            "title" : li.img.attrs.get('alt') if li.img else li.a.text.replace("\n", "").replace("\t","").replace("\r","") , 
            "date" : li.find(class_="date").text,
            "news_url" : li.a.attrs.get('href'),
            "image_url" :  li.img.attrs.get('src') if li.img else default_img,
            "category" : s }
            try :
                news_contents = get_news_contents(news_info['news_url'])
                news_info['contents'] = news_contents
                news_info_list.append(news_info)
            except Exception as e : 
                continue
        
        current_page += 1 
    
    print(s + " 분야 크롤링 완료")    
    return news_info_list

sid = ['100', '101', '102', '103', '104', '105']
default_url = "https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1="
df = pd.DataFrame()

for s in tqdm(sid) : 
    news = get_news_info(default_url, s)
    df = df.append(news)

df.to_csv('네이버뉴스_{}.csv'.format(date), encoding = "utf-8-sig")

20211115


  0%|          | 0/6 [00:00<?, ?it/s]

100 분야 크롤링 완료
101 분야 크롤링 완료
102 분야 크롤링 완료
103 분야 크롤링 완료
104 분야 크롤링 완료
105 분야 크롤링 완료


# 함수화

In [57]:
def get_preprocessing_data(path) : 
    data = pd.read_csv(path + '네이버뉴스_2021-11-15_06시16분.csv')
    data['title_contents'] = data['title'] + " " + data['contetns']
    data.drop([data.columns[0]], axis=1, inplace=True)
    data.drop(['date','image_url','title', 'contetns'], axis = 1, inplace = True)
    # 결측치 처리 
    data = data.fillna(" ")
    
    return data 

In [48]:
def make_doc2vec_data(data, column, t_document=False):
    data_doc = []
    for tag, doc in zip(data.index, data[column]):
        doc = doc.split(" ")
        data_doc.append(([tag], doc))
    if t_document:
        data = [TaggedDocument(words=text, tags=tag) for tag, text in data_doc]
        return data
    else:
        return data_doc

In [49]:
def make_doc2vec_models(tagged_data, tok, vector_size=128, window = 3, epochs = 40, min_count = 0, workers = 4):
    model = Doc2Vec(tagged_data, vector_size=vector_size, window=window, epochs=epochs, min_count=min_count, workers=workers)
    model.save(f'./{tok}_news_model.doc2vec')

In [84]:
# 두연님 model 
def make_word2vec_models() :
    model = Word2Vec(tokenized_data,  # 리스트 형태의 데이터
                 sg=1,                # 0: CBOW, 1: Skip-gram
                 vector_size=100,     # 벡터 크기
                 window=3,     # 고려할 앞뒤 폭(앞뒤 3단어)
                 min_count=3,  # 사용할 단어의 최소 빈도(3회 이하 단어 무시)
                 workers=4)    # 동시에 처리할 작업 수(코어 수와 비슷하게 설정)

In [50]:
def make_user_embedding(index_list, data_doc, model):
    user = []
    user_embedding = []
    for i in index_list:
        user.append(data_doc[i][0][0])
    for i in user:
        user_embedding.append(model.dv[i])
    user_embedding = np.array(user_embedding)
    user = np.mean(user_embedding, axis = 0)
    return user

In [76]:
def get_recommened_contents(user, data_doc, model):
    scores = []

    for tags, text in data_doc:
        trained_doc_vec = model.dv[tags[0]]
        scores.append(cosine_similarity(user.reshape(-1, 128), trained_doc_vec.reshape(-1, 128)))

    scores = np.array(scores).reshape(-1)
    scores = np.argsort(-scores)[:10]
    
    return data.loc[scores, :]

In [64]:
def view_user_history(data):
    return data[['title_contents', 'category']]

# Data Loading

In [65]:
path = "./"
input_data = get_preprocessing_data(path)
input_data

,news_url,category,title_contents
0,https://news.naver.com/main/read.naver?mode=LS...,100,엇갈린 부동산 정책…李 '보유세 강화' 尹 '1주택 종부세 비과세' (서울=뉴스1)...
1,https://news.naver.com/main/read.naver?mode=LS...,100,尹 “종부세 폭탄 걱정마라” 與 “시장 혼란 초래할 것” 국민의힘 윤석열 대선후보가...
2,https://news.naver.com/main/read.naver?mode=LS...,100,"“이준석, 가족 겁박”, “윤석열, 日에 면죄부”… 맹공 퍼붓는 與 더불어민주당은 ..."
3,https://news.naver.com/main/read.naver?mode=LS...,100,“언론 환경 너무 안 좋아”… 연일 언론 탓하는 이재명 더불어민주당 이재명 대선후보...
4,https://news.naver.com/main/read.naver?mode=LS...,100,"고민정, 블라인드 채용법 발의 알리려다 ‘모교 비하‘ 논란 더불어민주당 고민정 의원..."
...,...,...,...
525,https://news.naver.com/main/read.naver?mode=LS...,105,게임 즐기는 3040… 고가 마우스-헤드셋 잘나간다 이마트는 올해 1∼10월 가격 ...
526,https://news.naver.com/main/read.naver?mode=LS...,105,[최승진의 게임카페] '아케인'은 요란한 빈 수레가 아니다 리그오브레전드 유명 챔피...
527,https://news.naver.com/main/read.naver?mode=LS...,105,"2025년 미국 달 착륙 장면, 실시간 4K로 볼 수 있을까 국내 대학과 민간기업..."
528,https://news.naver.com/main/read.naver?mode=LS...,105,게임 즐기는 3040… 고가 마우스-헤드셋 잘나간다 이마트는 올해 1∼10월 가격 ...


# Recommend

In [66]:
data_doc_contents = make_doc2vec_data(input_data, 'title_contents')
data_doc_contents_tag = make_doc2vec_data(input_data, 'title_contents', t_document=True)

make_doc2vec_models(data_doc_contents_tag, tok=False)

model_contents = Doc2Vec.load('./False_news_model.doc2vec')

In [70]:
user_category_0 = input_data.loc[random.sample(data.loc[data.category == 100, :].index.values.tolist(), 10), : ]
user_category_1 = input_data.loc[random.sample(data.loc[data.category == 101, :].index.values.tolist(), 10), : ]
user_category_2 = input_data.loc[random.sample(data.loc[data.category == 102, :].index.values.tolist(), 10), : ]

view_user_history(user_category_1)

,title_contents,category
108,1년 연기? 공제 상향?… ‘가상자산 과세’ 흔드는 정치논리 가상자산 과세를 예정대...,101
115,세계 상장기업 올 3분기 순익 50%↑…美 급증·中 횡보 세계 주요 상장기업의 올해...,101
193,‘요소수·유가 쇼크’ 전기차·수소차 수요 폭발…“내연기관 종말 빨라지나” 대구에서 ...,101
150,"""우수 중소기업 360개사, 인천공항 면세점 입점"" (서울=뉴스1) 윤다정 기자 =...",101
129,허위연식 의심 타워크레인 188대 조사한다 [이데일리 하지나 기자] 국토교통부는 1...,101
179,"크래프톤 등 6개 종목, MSCI 한국 지수 편입 모건스탠리캐피털인터내셔널(MSC...",101
176,GC녹십자웰빙 ‘메타바이오틱스’ 특허 유산균 신제품 판매 본격화 GC녹십자웰빙은 유...,101
148,"'선글라스 맛집' 젠틀몬스터 신상…SSG닷컴, NFT 기술 보증서도 [서울=뉴시스]...",101
190,얼굴 인식해 카드 발급 내년 1분기부터 신용카드를 비대면(온라인)으로 발급받을 때...,101
198,"KT&G, 문화예술·지역사회 상생 프로그램 지원 나서 KT&G가 정부의 단계적 일상...",101


In [71]:
user_0 = make_user_embedding(user_category_0.index.values.tolist(), data_doc_contents, model_contents)
user_1 = make_user_embedding(user_category_1.index.values.tolist(), data_doc_contents, model_contents)
user_2 = make_user_embedding(user_category_2.index.values.tolist(), data_doc_contents, model_contents)

In [77]:
result_0 = get_recommened_contents(user_0, data_doc_contents, model_contents)
pd.DataFrame(result_0.loc[:, ])

,Unnamed: 0,title,date,news_url,image_url,category,contetns
19,19,"야구장 간 윤석열…""종부세 폭탄 없애겠다"" 공약도",26분전,https://news.naver.com/main/read.naver?mode=LS...,https://imgnews.pstatic.net/image/origin/422/2...,100,"'종합부동산세를 전면 재검토 하겠다며 구체적인 공약도 발표했는데, 본격적인 '표심잡..."
29,29,매타버스 탄 李… “NASA 같은 우주전략본부로 통합해야”,1시간전\t,https://news.naver.com/main/read.naver?mode=LS...,https://search.naver.com/search.naver?where=im...,100,이재명 더불어민주당 대선 후보가 ‘이재명의 매타버스(매주 타는 민생 버스)’ 전국 ...
56,56,야구 '직관'하며 젊은층 공략...김종인 출판기념회 참석,4시간전\t,https://news.naver.com/main/read.naver?mode=LS...,https://imgnews.pstatic.net/image/origin/052/2...,100,오늘은 총괄선대위원장으로 유력한 김종인 전 비대위원장의 출판기념회에 참석해 축사를 ...
17,17,윤석열 선대위 윤곽 '권한 분산'…따로 가는 홍준표,20분전,https://news.naver.com/main/read.naver?mode=LS...,https://imgnews.pstatic.net/image/origin/422/2...,100,국민의힘 선대위 구성을 둘러싼 잡음이 이어지는 가운데 윤석열 후보는 '실세' 총괄선...
1,1,尹 “종부세 폭탄 걱정마라” 與 “시장 혼란 초래할 것”,9분전,https://news.naver.com/main/read.naver?mode=LS...,https://imgnews.pstatic.net/image/origin/022/2...,100,국민의힘 윤석열 대선후보가 14일 종합부동산세 개편안의 청사진을 발표했다. 윤 후보...
16,16,"문 대통령, 오늘 윤석열 대선 후보에 축하난",19분전,https://news.naver.com/main/read.naver?mode=LS...,https://imgnews.pstatic.net/image/origin/055/2...,100,문재인 대통령은 오늘(15일) 이철희 청와대 정무수석을 통해 국민의힘 윤석열 대선후...
46,46,총괄 본부장 없애고 본부장 4~5명 둔다,2시간전\t,https://news.naver.com/main/read.naver?mode=LS...,https://search.naver.com/search.naver?where=im...,100,국민의힘 윤석열 대선 후보가 선거대책위원회에 총괄 선대본부장을 두지 않기로 가닥을 ...
18,18,"여, '종부세 재검토' 윤에 ""부자본색…불의한 주장""",23분전,https://news.naver.com/main/read.naver?mode=LS...,https://imgnews.pstatic.net/image/origin/422/2...,100,더불어민주당은 국민의힘 윤석열 대선후보가 종부세를 전면 재검토하겠다고 밝힌 것과 관...
33,33,"문대통령, 오늘 尹에 축하난…국민의힘 후보 선출 열흘만",1시간전\t,https://news.naver.com/main/read.naver?mode=LS...,https://imgnews.pstatic.net/image/origin/001/2...,100,(서울=연합뉴스) 박경준 기자 = 문재인 대통령은 15일 이철희 청와대 정무수석을 ...
23,23,"靑, 오늘 윤석열·김동연·안철수에 文대통령 축하 난 전달",46분전,https://news.naver.com/main/read.naver?mode=LS...,https://imgnews.pstatic.net/image/origin/421/2...,100,(서울=뉴스1) 박혜연 기자 = 청와대는 15일 윤석열 국민의힘 대선 후보에게 문재...


In [79]:
result_1 = get_recommened_contents(user_1, data_doc_contents, model_contents)
pd.DataFrame(result_1.loc[:, ])

,Unnamed: 0,title,date,news_url,image_url,category,contetns
197,97,"CJ제일제당, 업계 유일 7년연속 ‘DJSI 아시아·태평양 지수’ 등재",2시간전\t,https://news.naver.com/main/read.naver?mode=LS...,https://imgnews.pstatic.net/image/origin/005/2...,101,CJ제일제당은 ‘다우존스 지속가능경영지수’(DJSI) 평가에서 7년 연속 아시아·태...
176,76,GC녹십자웰빙 ‘메타바이오틱스’ 특허 유산균 신제품 판매 본격화,1시간전\t,https://news.naver.com/main/read.naver?mode=LS...,https://imgnews.pstatic.net/image/origin/081/2...,101,GC녹십자웰빙은 유산균 신제품인 ‘메타바이오틱스’의 본격적인 판매에 들어갔다고 14...
248,48,서울교육청-미용사중앙회 업무협약…미용분야 취업 지원 확대,17분전,https://news.naver.com/main/read.naver?mode=LS...,https://imgnews.pstatic.net/image/origin/421/2...,102,(서울=뉴스1) 장지훈 기자 = 서울시교육청은 15일 대한미용사중앙회와 업무협약을 ...
149,49,"롯데마트, 김장철 맞아 국내산 수육용 돼지고기 최대 30%↓",17분전,https://news.naver.com/main/read.naver?mode=LS...,https://imgnews.pstatic.net/image/origin/003/2...,101,[서울=뉴시스]김정현 기자 = 롯데마트는 오는 18일부터 24일까지 김장철을 맞아 ...
156,56,"한국 GM “전기차, 한국선 생산 안한다”",31분전,https://news.naver.com/main/read.naver?mode=LS...,https://imgnews.pstatic.net/image/origin/382/2...,101,한국GM이 국내 시장에서 판매되는 GM의 전기차 라인업을 2025년까지 10종으로 ...
134,34,인천공항 면세점에 중소기업 제품 전용판매장 3곳 개소,17분전,https://news.naver.com/main/read.naver?mode=LS...,https://imgnews.pstatic.net/image/origin/001/2...,101,(서울=연합뉴스) 박상돈 기자 = 중소벤처기업부는 15일 인천공항 면세점에 중소기업...
212,12,수도권 병상 가동률 '빨간불'‥오늘 2천 명 안팎 예상,13분전,https://news.naver.com/main/read.naver?mode=LS...,https://imgnews.pstatic.net/image/origin/214/2...,102,"어젯밤 9시 기준 전국의 코로나19 신규 확진자는 1천867명, 전날 같은 시각보다..."
150,50,"""우수 중소기업 360개사, 인천공항 면세점 입점""",17분전,https://news.naver.com/main/read.naver?mode=LS...,https://imgnews.pstatic.net/image/origin/421/2...,101,(서울=뉴스1) 윤다정 기자 = 인천국제공항 면세점 내에 우수한 우리 중소기업 36...
115,15,세계 상장기업 올 3분기 순익 50%↑…美 급증·中 횡보,14분전,https://news.naver.com/main/read.naver?mode=LS...,https://imgnews.pstatic.net/image/origin/374/2...,101,세계 주요 상장기업의 올해 3분기 순이익이 지난해 같은 기간에 비해 50% 늘어난 ...
165,65,SK에코플랜트 ‘중대 무사고 500일 달성 기념 행사’,1시간전\t,https://news.naver.com/main/read.naver?mode=LS...,https://imgnews.pstatic.net/image/origin/023/2...,101,SK에코플랜트가 지난 9일 서울 종로구 지플랜트 사옥에서 박경일 사장<사진>과 국내...
